# bronze.general.load_data.elt

## Overview
 
|Attribute|Details|
---|---|
|Created By|Rich Henry|
|Notebook Type|elt|
|Workflow Parent||
|Target Catalog|bronze|
|Target Database|general|
 
## Purpose
 
This notebook populates the bronze.general.<project name> table with any new rows from the TBD data input

## Revisions
 
|Revision Date|Developer|Comment|
---|---|---|
|2025-06-19|Rich Henry|Created|

In [0]:
%python

# Setting the environment.
environment = dbutils.secrets.get(scope = "workspace-details", key = "name")
spark.conf.set("sql.environment", environment)

# Setting the table names.
table_name = "bet_config"
stage_table_name = f"{environment}_silver.staging.{table_name}"
spark.conf.set("sql.table_name", table_name)
spark.conf.set("sql.stage_table_name", stage_table_name)


In [0]:
%sql
INSERT INTO
  ${sql.table_name} (
    game_spid,
    row_id,
    id,
    lines,
    bet_multiplier,
    bet_in_credits,
    bet_config_game_id,
    max_bet,
    is_wagerable,
    ways
  )
SELECT
  game_spid,
  row_id,
  id,
  lines,
  bet_multiplier,
  bet_in_credits,
  bet_config_game_id,
  max_bet,
  is_wagerable,
  ways
FROM
  ${sql.stage_table_name};
--Delete orphaned dim_bet_config_game rows
DELETE FROM
  dim_bet_config_game
WHERE
  bet_config_game_id NOT IN (
    SELECT
      bet_config_game_id
    FROM
      ${sql.table_name}
  );